In [1]:
import os
import sys
custom_paths = ['../src/', '../cifar-10.1-pytorch/', '../data-copying/', '../PyTorch_CIFAR10/']
for path in custom_paths:
    if os.path.abspath(path) not in sys.path:
        sys.path.insert(0, os.path.abspath(path))
        print(f"Added {os.path.abspath(path)} to path")

Added /work/cybai2020/PyTorch-StudioGAN/src to path
Added /work/cybai2020/PyTorch-StudioGAN/cifar-10.1-pytorch to path
Added /work/cybai2020/PyTorch-StudioGAN/data-copying to path
Added /work/cybai2020/PyTorch-StudioGAN/PyTorch_CIFAR10 to path


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import functools
from importlib import reload
from matplotlib import pyplot as plt
import scipy
from scipy import stats
from scipy.special import softmax
import os
import ast
import glob
import pickle
import yaml

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans 

import torch
from torch.nn import Parameter as P
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# import custom

import cifar10_models
reload(cifar10_models)

import cifar10_1
reload(cifar10_1)

from data_utils import cifair
reload(cifair)

from metrics.feat_extract_models import alexnet, inception, vgg
reload(alexnet)
reload(inception)
reload(vgg)

from utils import load_checkpoint
reload(load_checkpoint)

from metrics.FID import calculate_frechet_distance
from models.reID import gan_proj_ft_net

import misc
reload(misc)

import data_copying_tests
reload(data_copying_tests)

from run_infos import runs

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = 'cuda'
torch.backends.cudnn.benchmark = True

In [4]:
# proj_model = inception.WrappedInception3().eval().to(device)

with open('../Person_reID_baseline_pytorch/model/ft_ResNet50/opts.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
proj_model = gan_proj_ft_net(config['nclasses'], stride=config['stride'])
proj_model.load_state_dict(torch.load('../Person_reID_baseline_pytorch/model/ft_ResNet50/net_last.pth'))
proj_model = proj_model.eval().to(device)

In [ ]:
for runs in [runs['CIFAR10']['sagan'], runs['CIFAR10']['diffaug']]:
    for runnames in runs.values():
        for runname in runnames:
            sampler = misc.wrapper_best(runname, dset_name='CIFAR10', device=device)
            misc.build_cache(runname, sampler, cache_dir='/tmp2/cybai/studiogan/cache', 
                             conditional=True, proj_model=proj_model, verbose=True)

cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_04_15_20_57/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_09_00_54_36/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_10_10_00_29/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_17_13_42_52/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_09_00_57_09/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_11_10_38_54/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_18_13_59_54/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAGAN-train-2021_03_06_00_46_59/best.npz
(50000, 3, 32, 32) (50000,) (50000, 2048)
cache to /tmp2/cybai/studiogan/cache/SAG

In [5]:
for runnames in runs['CELEBA-HQ_128']['biggan'].values():
    for runname in runnames:
        cache_path = os.path.join('/work/cybai2020/cache', runname, 'best.npz')
        if os.path.isfile(cache_path):
            print(f"{cache_path} already exists..")
            continue
        sampler = misc.wrapper_best(runname, dset_name='CELEBA-HQ_128', device=device)
        misc.build_cache(runname, sampler, cache_dir='/work/cybai2020/cache', 
                         conditional=False, proj_model=proj_model, avoid_list=['X', 'Y'], verbose=True)

/work/cybai2020/cache/BigGAN-train-2021_05_13_11_03_03/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_15_23_38_06/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_15_23_52_59/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_17_01_47_45/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_13_14_44_28/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_15_23_59_20/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_16_00_20_18/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_17_18_43_11/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_12_10_12_32/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_14_21_06_04/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_16_00_01_52/best.npz already exists..
/work/cybai2020/cache/BigGAN-train-2021_05_17_02_09_46/best.npz already exists..
/work/cybai2020/cache/BigGAN